# Data Science with Neo4j Using Yelp Data

### Module 2: Segmentation and Community Detection (Work in Progress)

Goal: Find communities based on categories of restaurants users review      

Algorithm: Label Propagation

High Level Approach:
      
- Subset user base to those who have reviewed restaurants in Toronto 
- Include stats to show there's enough users and reviews within this sample
- Leverage Jesús Barrasa's taxonomy post to identify restaurant sub-categories: https://jbarrasa.com/2017/03/31/quickgraph5-learning-a-taxonomy-from-your-tagged-data/
- Create weights between users to form tighter communities
- Draw parallels with traditional data science clustering approaches (eg: k-means, kNN)

In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4jneo4j"))

### Part 1: Find Restaurant Sub-categories

In [16]:
%%time
# find average number of categories a business can belong to, as well as min and max
query = """
        MATCH (b:Business)
        WITH size((b)-[:IN_CATEGORY]->()) AS count
        RETURN AVG(count) AS avgCat,  MIN(count) AS minCount, MAX(count) AS maxCat
        """

with driver.session() as session:
    result = session.run(query)

for row in result:
    print(row)

<Record avgCat=3.826724562629598 minCount=1 maxCat=36>
CPU times: user 2.21 ms, sys: 2.41 ms, total: 4.62 ms
Wall time: 1.5 s


In [73]:
# 
# note: look at distribution of categories a business belongs to in order to determine threshold to use. for now, use 5.
#

In [17]:
%%time
# compute co-occurence index between two categories. create relationship between two categories using co-occurence index
query = """
        MATCH (c:Category) WHERE SIZE((c)<-[:IN_CATEGORY]-()) > 5 // threshold - want more than 5 businesses in a cat
        WITH c, size((c)<-[:IN_CATEGORY]-()) as totalCount
        MATCH (c)<-[:IN_CATEGORY]-(b)-[:IN_CATEGORY]->(relatedCategory)
        WITH c, relatedCategory, toFloat(count(b)) / totalCount AS coocIndex
        CREATE (c)-[:CO_OCCURS {index: coocIndex }]->(relatedCategory)
        """

with driver.session() as session:
    result = session.run(query)
    

CPU times: user 2.63 ms, sys: 4.86 ms, total: 7.48 ms
Wall time: 14.3 s


In [48]:
%%time
# look at values for co-occurrence index
query = """
        MATCH (c1:Category)-[r:CO_OCCURS]->(c2:Category)
        WHERE r.index > 0
        RETURN c1.name, c2.name, r.index
        ORDER BY r.index DESCENDING
        """

with driver.session() as session:
    result = session.run(query)

index_df = pd.DataFrame([dict(record) for record in result])

CPU times: user 2.47 s, sys: 54.7 ms, total: 2.52 s
Wall time: 3.78 s


In [49]:
index_df

,c1.name,c2.name,r.index
0,Hair Stylists,Hair Salons,1.000000
1,Hair Stylists,Beauty & Spas,1.000000
2,Hair Salons,Beauty & Spas,1.000000
3,Men's Hair Salons,Hair Salons,1.000000
4,Men's Hair Salons,Beauty & Spas,1.000000
5,Blow Dry/Out Services,Beauty & Spas,1.000000
6,Blow Dry/Out Services,Hair Salons,1.000000
7,Hair Extensions,Hair Salons,1.000000
8,Hair Extensions,Beauty & Spas,1.000000
9,Departments of Motor Vehicles,Public Services & Government,1.000000


In [47]:
%%time
## case 1:  bi-directional co-occurrence is 1
query = """
        MATCH (c1)-[co1:CO_OCCURS {index : 1}]->(c2),
        (c2)-[co2:CO_OCCURS { index: 1}]->(c1)
        WHERE ID(c1) > ID(c2)
        MERGE (c1)-[:SAME_AS]-(c2)
        """

with driver.session() as session:
    result = session.run(query)

CPU times: user 2.75 ms, sys: 2.99 ms, total: 5.74 ms
Wall time: 18.2 s


In [54]:
%%time
# look at categories that are the same 
query = """
        MATCH (c1:Category)-[r:SAME_AS]->(c2:Category)
        RETURN c1.name, c2.name
        """

with driver.session() as session:
    result = session.run(query)

same_df = pd.DataFrame([dict(record) for record in result])

CPU times: user 2.83 ms, sys: 2.34 ms, total: 5.17 ms
Wall time: 9.98 ms


In [55]:
same_df  

,c1.name,c2.name
0,Mags,Books
1,Music & Video,Mags
2,Music & Video,Books
3,Wine & Spirits,Beer
4,Vintage & Consignment,Used
5,Trusts,Wills
6,& Probates,Trusts
7,& Probates,Wills


only 7 records returned above

In [57]:
%%time
query = """
        MATCH (c1)-[co1:CO_OCCURS]->(c2), 
              (c2)-[co2:CO_OCCURS]->(c1)
        WHERE ID(c1) > ID(c2) 
          AND co1.index = 1 
          AND co2.index < 1 
        MERGE (c1)-[:NARROWER_THAN]->(c2)
        """

with driver.session() as session:
    result = session.run(query)

CPU times: user 2.12 ms, sys: 3.13 ms, total: 5.25 ms
Wall time: 11.7 s


In [58]:
%%time
# look at categories that are NARROWER_THAN other categories
query = """
        MATCH (c1:Category)-[r:NARROWER_THAN]->(c2:Category)
        RETURN c1.name, c2.name
        """

with driver.session() as session:
    result = session.run(query)

narrower_df = pd.DataFrame([dict(record) for record in result])

CPU times: user 44.1 ms, sys: 3.58 ms, total: 47.7 ms
Wall time: 59.4 ms


In [59]:
narrower_df

,c1.name,c2.name
0,Men's Hair Salons,Hair Salons
1,Blow Dry/Out Services,Hair Salons
2,Hair Extensions,Hair Salons
3,Bars,Nightlife
4,Italian,Restaurants
5,Property Management,Home Services
6,Property Management,Real Estate
7,Auto Detailing,Automotive
8,Breakfast & Brunch,Restaurants
9,Gluten-Free,Restaurants


In [63]:
%%time
# check number of transitive relationships
query = """
        MATCH (c1)-[:NARROWER_THAN*2..]->(c3), 
        (c1)-[d:NARROWER_THAN]->(c3)
        RETURN COUNT(d) AS  num_transitive_relationships
        """

with driver.session() as session:
    result = session.run(query)
    
for row in result:
    print(row)

<Record num_transitive_relationships=0>
CPU times: user 2.84 ms, sys: 3.34 ms, total: 6.18 ms
Wall time: 12.2 s


In [ ]:
%%time
# since there are no transitive relationships, no need to delete redundant relationships
#query = """
#        MATCH (c1)-[:NARROWER_THAN*2..]->(c3), 
#        (c1)-[d:NARROWER_THAN]->(c3)
#        DELETE d
#        """

with driver.session() as session:
    result = session.run(query)

In [67]:
%%time
# get a list of categories that are narrower than the "Restaurants" category
query = """
        MATCH (c1:Category{name: "Restaurants"})<-[:NARROWER_THAN]-(c2:Category)
        RETURN c1.name, c2.name
        """

with driver.session() as session:
    result = session.run(query)

restaurants_narrower_df = pd.DataFrame([dict(record) for record in result])

CPU times: user 8.24 ms, sys: 3.21 ms, total: 11.5 ms
Wall time: 70.3 ms


In [68]:
restaurants_narrower_df

,c1.name,c2.name
0,Restaurants,Hawaiian
1,Restaurants,Cheesesteaks
2,Restaurants,Middle Eastern
3,Restaurants,Tapas Bars
4,Restaurants,Himalayan/Nepalese
5,Restaurants,Live/Raw Food
6,Restaurants,Cajun/Creole
7,Restaurants,Fish & Chips
8,Restaurants,Poutineries
9,Restaurants,Irish


### Part 2: Label Propagation to Detect Communities

In [ ]:
%%time
# before running label propagation, define similarity between user pairs using Jaccard index
# this will be used as the weight for Label Propagation
# numerator: count of categories that they've reviewed together
# denominator: count of categories between the two of them
# criteria: business has to be restaurant in Toronto
# for now, keep calculation simple.  room for improvement later (ie: take into account # businesses in each category)

query = """
        // Get user pairs and count of distinct restaurant categories that they have both reviewed
        MATCH (u1:User)-[:WROTE]->(r1:Review)-[:REVIEW_OF]->(b1:Business)-[:IN_CATEGORY]->(c:Category),
              (u2:User)-[:WROTE]->(r2:Review)-[:REVIEW_OF]->(b2:Business)-[:IN_CATEGORY]->(c),
              (b1)-[:IN_CITY]->(:City {name: "Toronto"})<-[:IN_CITY]-(b2),
              (c)-[:NARROWER_THAN]->(:Category{name: "Restaurants"})
        WHERE u1 <> u2
        RETURN u1, u2, COUNT(DISTINCT c) as intersection_count

        // Get count of all the distinct categories that they have reviewed between them
        //MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEW_OF]->(b:Business)-[:IN_CATEGORY]->(c:Category),
        //WHERE u in [u1, u2]
        //WITH u1, u2, intersection_count, COUNT(DISTINCT c) as union_count

        // Compute Jaccard index and store it as relationship between u1 and u2
        //WITH u1, u2, intersection_count, union_count, (intersection_count*1.0/union_count) as jaccard_index
        //CREATE (u1)-[s:SIMILAR_TO]-(u2)
        //SET s.jaccard_index = jaccard_index
        """

with driver.session() as session:
    result = session.run(query)

In [ ]:
jaccard_df = pd.DataFrame([dict(record) for record in result])

In [92]:
%%time
# incorporate weights between user pairs in order to create tighter communities
# weightProperty - Jaccard index
# subset to Restaurants in Toronto only
query = """
        CALL algo.labelPropagation(
            'MATCH (u:User)-[:WROTE]->(:Review)-[:REVIEW_OF]->(:Business)-[:IN_CITY]->(:City {name:"Toronto"}) RETURN id(u) as id',   
            
            'MATCH (u1:User)-[:WROTE]->(r1:Review)-[:REVIEW_OF]->(b1:Business)-[:IN_CATEGORY]->(c:Category),
                   (u2:User)-[:WROTE]->(r2:Review)-[:REVIEW_OF]->(b2:Business)-[:IN_CATEGORY]->(c),
                   (b1)-[:IN_CITY]->(:City {name: "Toronto"})<-[:IN_CITY]-(b2),
                   (:Category{name: "Restaurants"})<-[:NARROWER_THAN]-(c)

             RETURN DISTINCT id(u1) as source, id(u2) as target', 
             'OUTGOING',
            {graph: 'cypher', iterations:10, partitionProperty:'partition', write: true})
        YIELD nodes, iterations, loadMillis, computeMillis, writeMillis, write, partitionProperty;
        """

with driver.session() as session:
    result = session.run(query)
    
for row in result:
    print(row)

<Record nodes=6764792 iterations=10 loadMillis=29382 computeMillis=25497 writeMillis=39893 write=True partitionProperty='partition'>
CPU times: user 6.12 ms, sys: 9.46 ms, total: 15.6 ms
Wall time: 1min 34s


In [96]:
%%time
# check number of nodes
query = """
        MATCH (n)
        RETURN count(n)
        """

with driver.session() as session:
    result = session.run(query)
    
for row in result:
    print(row)

<Record count(n)=6764792>
CPU times: user 2.14 ms, sys: 2.71 ms, total: 4.85 ms
Wall time: 12.9 ms


In [99]:
%%time
# check number of qualifying user nodes (ie: users who reviewed businesses in Toronto)
query = """
        MATCH (u:User)-[:WROTE]->(:Review)-[:REVIEW_OF]->(:Business)-[:IN_CITY]->(:City {name:"Toronto"})
        RETURN count(u)
        """

with driver.session() as session:
    result = session.run(query)
    
for row in result:
    print(row)

<Record count(u)=430985>
CPU times: user 2.89 ms, sys: 5.25 ms, total: 8.15 ms
Wall time: 6.56 s


In [100]:
%%time
# check number of communities
query = """
        MATCH (u:User)
        RETURN count(DISTINCT u.partition)
        """

with driver.session() as session:
    result = session.run(query)

for row in result:
    print(row)

<Record count(DISTINCT u.partition)=872>
CPU times: user 2.39 ms, sys: 3.24 ms, total: 5.63 ms
Wall time: 1.99 s


In [101]:
%%time
# check number of business partitions????
query = """
        MATCH (b:Business)
        RETURN count(DISTINCT b.partition)
        """

with driver.session() as session:
    result = session.run(query)

for row in result:
    print(row)

<Record count(DISTINCT b.partition)=196>
CPU times: user 2.04 ms, sys: 3.18 ms, total: 5.23 ms
Wall time: 358 ms


In [ ]:
%%time
# remove partition
query = """
        MATCH (n:Review)
        REMOVE n.partition 
        RETURN n
        """

with driver.session() as session:
    result = session.run(query)